# 02b - Torus Segmentation Demo

This notebook demonstrates mesh segmentation using a torus geometry. The torus has genus = 1 (one hole), which creates multiple segments in middle slices.

**Part of the GenCoMo Tutorial Series** - [Return to Index](01_tutorial_index.ipynb)

In [ ]:
import numpy as np
import trimesh
from gencomo import create_torus_mesh, MeshSegmenter, visualize_mesh_3d

## Create and Visualize Torus

In [ ]:
# Create a torus mesh
torus = create_torus_mesh(major_radius=3.0, minor_radius=1.0, major_segments=24, minor_segments=12)

# Rotate so the torus lies on its side (ensures slices through the hole)
torus.apply_transform(trimesh.transformations.rotation_matrix(
    angle=np.pi / 2,
    direction=[1, 0, 0],
    point=torus.centroid
))

print(f"Torus properties:")
print(f"  Volume: {torus.volume:.3f}")
print(f"  Surface area: {torus.area:.3f}")
print(f"  Z-bounds: {torus.bounds[:, 2]}")

# Visualize the original torus
fig = visualize_mesh_3d(torus, title="Original Torus", backend="plotly")
fig.show()

## Segment the Mesh

In [ ]:
# Create segmenter and segment the mesh
segmenter = MeshSegmenter()
segments = segmenter.segment_mesh(torus, slice_width=0.8, min_volume=0.1)

print(f"Segmentation complete!")
print(f"Total segments: {len(segments)}")
print(f"Total slices: {len(segmenter.slices)}")

## Analyze Segments

In [ ]:
# Show segments per slice - should reveal multiple segments in middle slices
print("Segments per slice:")
for i in range(len(segmenter.slices)):
    slice_segments = segmenter.get_segments_in_slice(i)
    print(f"  Slice {i}: {len(slice_segments)} segments")
    if len(slice_segments) > 1:
        print(f"    -> Multiple segments detected (due to torus hole)")

# Analyze connectivity
connected_components = segmenter.get_connected_components()
print(f"\nConnected components: {len(connected_components)}")
for i, component in enumerate(connected_components):
    print(f"  Component {i+1}: {len(component)} segments")

# Get statistics
stats = segmenter.compute_segmentation_statistics()
print(f"\nTotal volume: {stats['volume_stats']['total']:.4f}")
print(f"Mean segment volume: {stats['volume_stats']['mean']:.4f}")

## 3D Connectivity Visualization

In [ ]:
# Visualize connectivity in 3D space - should show ring structure
segmenter.visualize_connectivity_graph_3d(backend="plotly")

## Summary

The torus segmentation demonstrates:
- **Complex topology**: Middle slices contain multiple segments due to the hole
- **Ring connectivity**: Segments form a circular arrangement in 3D space
- **Genus = 1 handling**: Algorithm correctly identifies separate connected components within slices
- **Spatial relationships**: 3D visualization reveals the donut structure

This showcases the algorithm's ability to handle complex topologies with holes.